In [ ]:
!sudo apt-get install swig3.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig3.0
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,109 kB of archives.
After this operation, 5,555 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 swig3.0 amd64 3.0.12-2.2ubuntu1 [1,109 kB]
Fetched 1,109 kB in 0s (8,295 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package s

In [ ]:
!sudo pip install jamspell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jamspell: filename=jamspell-0.0.12-cp310-cp310-linux_x86_64.whl size=1903642 sha256=e77dd86b092ad073ae9617b52baa4cfe91b87d2f5d43388d74e1a0eced2cd490
  Stored in directory: /root/.cache/pip/wheels/95/af/d3/fb57122fda2851ad5972ce80640001c568dcffdf16bc9ddb17
Successfully built jamspell


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%cd gdrive/MyDrive/TA\ Bareng\ Bareng/Laporan\ Mingguan\ TA\ Harakan\ Julian\ Fausta/Minggu\ 15\ (Final)/Ekstraksi/

/content/gdrive/.shortcut-targets-by-id/1mVs_asXEy-dFb5kdY3WcjA5GRJ2ZyRQ_/TA Bareng Bareng/Laporan Mingguan TA Harakan Julian Fausta/Minggu 15 (Final)/Ekstraksi


In [ ]:
!ls

'0. Korpus Entri'
'10. Morphology'
'11. Parallel Corpus'
'12. Parallel Corpus - Spelling Checker'
'1. Preprocessing - Header & Footer'
'2. Preprocessing - Generalisasi Polisemi'
'3. Preprocessing - Replace Kata Singkatan'
'4. Preprocessing - Remove Anomali Entri'
'5. Kategorisasi Main Entry'
'6. Pemecahan Definisi Lema'
'7. Ekstraksi Informasi Sumber Daya NLP'
'8. Cleaning Informasi Sumber Daya NLP'
'9. Bilingual Lexicon'
'X. Data Pendukung'
'Y. Hasil Ekstraksi'


In [ ]:
import pandas as pd

lookup_df = pd.read_csv('12. Parallel Corpus - Spelling Checker/LookupIsFromIndonesia.csv')
print(lookup_df.shape)

(43, 2)


In [ ]:
type(lookup_df['is_from_indonesia'][0])

numpy.int64

In [ ]:
import jamspell

jsp = jamspell.TSpellCorrector()
assert jsp.LoadLangModel('12. Parallel Corpus - Spelling Checker/model_subtitles.bin')

In [ ]:
def get_is_from_indonesia(input_path):
    for index, row in lookup_df.iterrows():
        if row['kamus'] in input_path:
            return True, row['is_from_indonesia']
    return False, -1

In [ ]:
def sentence_to_list(sentence):
    word_list = []
    word = ''
    for c in sentence.lower():
        if c.isalpha():
            word += c
        else:
            word_list.append(word)
            word = ''
    if word:
        word_list.append(word)
    return word_list

In [ ]:
import pandas as pd

changes_list = []
# kamus_list = ['18', '54', '68', '71', '89']

def postprocess(input_path, output_path):
    # found = False
    # for kamus in kamus_list:
    #     if kamus in input_path:
    #         found = True
    # if not found:
    #     return
    print(input_path)
    print(output_path)
    df = pd.read_csv(input_path)
    kalimat_asal_result_list = []
    kalimat_tujuan_result_list = []
    words_modified = 0
    words_total = 0
    sentences_modified = 0
    sentences_total = 0
    success, is_from_indonesia = get_is_from_indonesia(input_path)

    if not success:
        return

    for index, row in df.iterrows():
        kalimat_asal = str(row['kalimat_asal']).lower()
        kalimat_tujuan = str(row['kalimat_tujuan']).lower()
        list_kata_asal = sentence_to_list(kalimat_asal)
        list_kata_tujuan = sentence_to_list(kalimat_tujuan)

        kalimat_asal_result = kalimat_asal
        kalimat_tujuan_result = kalimat_tujuan
        list_kata_asal_result = list_kata_asal
        list_kata_tujuan_result = list_kata_tujuan

        if is_from_indonesia:
            kalimat_asal_result = jsp.FixFragment(kalimat_asal)
            list_kata_asal_result = sentence_to_list(kalimat_asal_result)
            sentences_total += 1
            if kalimat_asal != kalimat_asal_result:
                sentences_modified += 1
            words_total += len(list_kata_asal)
            for i in range(len(list_kata_asal)):
                if list_kata_asal[i] != list_kata_asal_result[i]:
                    words_modified += 1
                    changes_list.append((list_kata_asal[i], list_kata_asal_result[i]))
        else:
            kalimat_tujuan_result = jsp.FixFragment(kalimat_tujuan)
            list_kata_tujuan_result = sentence_to_list(kalimat_tujuan_result)
            sentences_total += 1
            if kalimat_tujuan != kalimat_tujuan_result:
                sentences_modified += 1
            words_total += len(list_kata_tujuan)
            for i in range(len(list_kata_tujuan)):
                if list_kata_tujuan[i] != list_kata_tujuan_result[i]:
                    words_modified += 1
                    changes_list.append((list_kata_tujuan[i], list_kata_tujuan_result[i]))

        kalimat_asal_result_list.append(kalimat_asal_result)
        kalimat_tujuan_result_list.append(kalimat_tujuan)

    result_df = pd.DataFrame(
        list(zip(kalimat_asal_result_list, kalimat_tujuan_result_list)),
        columns=['kalimat_asal', 'kalimat_tujuan'])

    print('percentage words modified', 100.0 * words_modified / words_total)
    print('percentage sentences modified', 100.0 * sentences_modified / sentences_total)
    print(result_df.shape)
    result_df.to_csv(output_path, index=False)

In [ ]:
import os

dir_path = '11. Parallel Corpus'

for file in os.listdir(dir_path):
    if file[-4:] == '.csv':
        print(file)
        input_path = dir_path + '/' + file
        output_path = '12. Parallel Corpus - Spelling Checker/' + file
        postprocess(input_path, output_path)

12_Parcor.csv
11. Parallel Corpus/12_Parcor.csv
12. Parallel Corpus - Spelling Checker/12_Parcor.csv
percentage words modified 17.00272008980614
percentage sentences modified 55.19250780437045
(4805, 2)
10_Parcor.csv
11. Parallel Corpus/10_Parcor.csv
12. Parallel Corpus - Spelling Checker/10_Parcor.csv
percentage words modified 14.97458621139059
percentage sentences modified 65.26822558459422
(2908, 2)
18_Parcor.csv
11. Parallel Corpus/18_Parcor.csv
12. Parallel Corpus - Spelling Checker/18_Parcor.csv
percentage words modified 11.237420797614611
percentage sentences modified 32.56212510711225
(2334, 2)
21_Parcor.csv
11. Parallel Corpus/21_Parcor.csv
12. Parallel Corpus - Spelling Checker/21_Parcor.csv
percentage words modified 12.115033801910077
percentage sentences modified 42.42424242424242
(2013, 2)
20_Parcor.csv
11. Parallel Corpus/20_Parcor.csv
12. Parallel Corpus - Spelling Checker/20_Parcor.csv
percentage words modified 3.5552536231884058
percentage sentences modified 17.9382644

In [ ]:
from collections import Counter

changes_counter = Counter(changes_list)

changes_counter.most_common(10)

[(('sebagianya', 'sebagainya'), 271),
 (('adv', 'ada'), 27),
 (('iru', 'itu'), 23),
 (('sagu', 'satu'), 19),
 (('mi', 'di'), 15),
 (('kedil', 'kecil'), 12),
 (('nedda', 'ledda'), 9),
 (('seget', 'segel'), 8),
 (('gurnita', 'gurita'), 7),
 (('senek', 'nenek'), 7)]